In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# Scraping

In [15]:
raw_html = requests.get("https://www.listchallenges.com/completely-random-movie-list").text

html = BeautifulSoup(raw_html)

movies = [ 
    movie.get_text(strip=True).split(' (')[0] 
    for movie in html.select('div.item-name') 
]

In [16]:
movies_metadata = [
    json.loads(requests.get('http://www.omdbapi.com/?t={}&apikey=f6adbb42'.format(movie)).text)
    for movie in movies
]

In [17]:
df = pd.DataFrame(movies_metadata)

# Test de l'API

In [18]:
data = {
    'movies': json.dumps(movies)
}

df = pd.DataFrame(requests.post("http://40.89.160.129:5000/get_movie_metadata", data).json())

json_data = requests.post("http://40.89.160.129:5000/get_movie_metadata", data).json()

In [19]:
data = {
    'model_name': 'model_test',
    'data': json.dumps(json_data)
}

requests.post("http://40.89.160.129:5000/train_model", data).text

'SUCCESS'

In [20]:
def parse_box_office(raw_box_office):
    if(raw_box_office != 'N/A'):
        return raw_box_office.replace('$', '').replace(',', '')
    else:
        return None

df['BoxOffice'] = df['BoxOffice'].apply(parse_box_office).dropna()
df['USA?'] = df['Country'].apply(lambda country: 1 if (country == 'USA') else 0)
df['Genre'] = df['Genre'].apply(lambda genre: genre.split(',')[0])
df = pd.get_dummies(df, columns=['Genre'], drop_first=True)
df = df.dropna()
X = df[['BoxOffice', 'USA?'] + [col for col in df.columns if 'Genre_' in col]]


data = {
    'model_name': 'model_test',
    'data': json.dumps(X.to_dict())
}

requests.post("http://40.89.160.129:5000/predict", data).json()

[7.313236881920363,
 6.31458139695239,
 6.200000000000001,
 8.005072099261923,
 7.423194375087286,
 6.634326805640287,
 7.286896081721812,
 8.21293665956683,
 7.479037470347755,
 6.689202661985279,
 7.594927900738078,
 6.885418603047615,
 7.619992997371452,
 7.2362449458056055,
 7.181351419668872,
 6.843259977948908,
 6.842565767237795,
 7.08165900297722,
 7.114658528651754,
 8.100000000000001,
 6.810797338014724,
 7.963568742992351,
 8.467070343061716]